In [ ]:
%%capture --no-stderr
# %pip install -U langgraph langsmith langchain_anthropic --ignore-installed

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\GINROD\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\zstandard\\backend_c.cp313-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:

import os

ANTHROPIC_API_KEY = "ANTHROPIC_API_KEY"

os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

TAVILY_API_KEY = "TAVILY_API_KEY"

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [4]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=2)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
llm = llm.bind_tools(tools=[tool])

def chatbot(state):
    return {"messages": [llm.invoke(state["messages"])]}

CHATBOT = "chatbot"

graph_builder.add_node(CHATBOT, chatbot)

tool_node = ToolNode(tools=[tool])
TOOLS = "tools"

graph_builder.add_node(TOOLS, tool_node)
graph_builder.add_conditional_edges(CHATBOT, tools_condition)

graph_builder.add_edge(TOOLS, CHATBOT)
graph_builder.set_entry_point(CHATBOT)

In [ ]:
graph = graph_builder.compile(checkpointer=memory)